<a href="https://colab.research.google.com/github/nayvti380/Data-Science/blob/main/Data_Science_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, roc_curve, auc, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
url = r"https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
column_names = [
    "age", "sex", "cp", "trestbps", "chol", "fbs", "restecg",
    "thalach", "exang", "oldpeak", "slope", "ca", "thal", "target"
]
df = pd.read_csv(url, names=column_names)
print(df.head())

    age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  \
0  63.0  1.0  1.0     145.0  233.0  1.0      2.0    150.0    0.0      2.3   
1  67.0  1.0  4.0     160.0  286.0  0.0      2.0    108.0    1.0      1.5   
2  67.0  1.0  4.0     120.0  229.0  0.0      2.0    129.0    1.0      2.6   
3  37.0  1.0  3.0     130.0  250.0  0.0      0.0    187.0    0.0      3.5   
4  41.0  0.0  2.0     130.0  204.0  0.0      2.0    172.0    0.0      1.4   

   slope   ca thal  target  
0    3.0  0.0  6.0       0  
1    2.0  3.0  3.0       2  
2    2.0  2.0  7.0       1  
3    3.0  0.0  3.0       0  
4    1.0  0.0  3.0       0  


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 297 entries, 0 to 301
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       297 non-null    float64
 1   sex       297 non-null    float64
 2   cp        297 non-null    float64
 3   trestbps  297 non-null    float64
 4   chol      297 non-null    float64
 5   fbs       297 non-null    float64
 6   restecg   297 non-null    float64
 7   thalach   297 non-null    float64
 8   exang     297 non-null    float64
 9   oldpeak   297 non-null    float64
 10  slope     297 non-null    float64
 11  ca        297 non-null    float64
 12  thal      297 non-null    float64
 13  target    297 non-null    int64  
dtypes: float64(13), int64(1)
memory usage: 34.8 KB


In [ ]:
for coluna in df.columns:
    print(f"Coluna: {coluna}")
    print(f"Valores únicos: {df[coluna].unique()}")

Coluna: age
Valores únicos: [63. 67. 37. 41. 56. 62. 57. 53. 44. 52. 48. 54. 49. 64. 58. 60. 50. 66.
 43. 40. 69. 59. 42. 55. 61. 65. 71. 51. 46. 45. 39. 68. 47. 34. 35. 29.
 70. 77. 38. 74. 76.]
Coluna: sex
Valores únicos: [1. 0.]
Coluna: cp
Valores únicos: [1. 4. 3. 2.]
Coluna: trestbps
Valores únicos: [145. 160. 120. 130. 140. 172. 150. 110. 132. 117. 135. 112. 105. 124.
 125. 142. 128. 170. 155. 104. 180. 138. 108. 134. 122. 115. 118. 100.
 200.  94. 165. 102. 152. 101. 126. 174. 148. 178. 158. 192. 129. 144.
 123. 136. 146. 106. 156. 154. 114. 164.]
Coluna: chol
Valores únicos: [233. 286. 229. 250. 204. 236. 268. 354. 254. 203. 192. 294. 256. 263.
 199. 168. 239. 275. 266. 211. 283. 284. 224. 206. 219. 340. 226. 247.
 167. 230. 335. 234. 177. 276. 353. 243. 225. 302. 212. 330. 175. 417.
 197. 198. 290. 253. 172. 273. 213. 305. 216. 304. 188. 282. 185. 232.
 326. 231. 269. 267. 248. 360. 258. 308. 245. 270. 208. 264. 321. 274.
 325. 235. 257. 164. 141. 252. 255. 201. 222. 260. 182.

In [ ]:
print(df["ca"].value_counts())
print(df["thal"].value_counts())
print(df.shape)

ca
0.0    176
1.0     65
2.0     38
3.0     20
?        4
Name: count, dtype: int64
thal
3.0    166
7.0    117
6.0     18
?        2
Name: count, dtype: int64
(303, 14)


In [ ]:
df = df.replace('?', np.nan)
df = df.dropna()

In [ ]:
df['ca'] = pd.to_numeric(df['ca'])
df['thal'] = pd.to_numeric(df['thal'])
# A coluna 'target' indica a presença de doença cardíaca (0 = não, >0 = sim).
# Vamos binarizar para 0 (não) e 1 (sim).
df['target'] = df['target'].apply(lambda x: 1 if x > 0 else 0)

# Matriz de Confusão

In [ ]:
x = df.drop('target', axis=1)
y = df['target']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)
print(f"Proporção de classes no treino:\n{y_train.value_counts(normalize=True)}")
print(f"Proporção de classes no teste:\n{y_test.value_counts(normalize=True)}")

Proporção de classes no treino:
target
0    0.541063
1    0.458937
Name: proportion, dtype: float64
Proporção de classes no teste:
target
0    0.533333
1    0.466667
Name: proportion, dtype: float64


In [ ]:

model = LogisticRegression(solver='liblinear', random_state=42) # 'liblinear' é bom para datasets menores
model.fit(x_train, y_train)
model.score(x_train, y_train)

0.8357487922705314

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
# Ajuda a visualizar True Positives (TP), True Negatives (TN), False Positives (FP), False Negatives (FN)
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
print(f"""
TN|FP
FN|TP
""")

[[45  3]
 [ 7 35]]

TN|FP
FN|TP



In [ ]:
# Precision: Dos que o modelo previu como positivos, quantos são realmente positivos?
# TP / (TP + FP)
precision = precision_score(y_test, y_pred)
print(f"\nPrecision: {precision:.4f}")
print("Interpretação: Dos pacientes que nosso modelo previu terem doença cardíaca, "
      f"{precision*100:.2f}% realmente têm a doença.")


Precision: 0.9211
Interpretação: Dos pacientes que nosso modelo previu terem doença cardíaca, 92.11% realmente têm a doença.


In [ ]:
# Recall (Sensibilidade): Dos que são realmente positivos, quantos o modelo conseguiu identificar?
# TP / (TP + FN)
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.4f}")
print("Interpretação: De todos os pacientes que REALMENTE têm doença cardíaca, "
      f"nosso modelo identificou corretamente {recall*100:.2f}% deles.")

Recall: 0.8333
Interpretação: De todos os pacientes que REALMENTE têm doença cardíaca, nosso modelo identificou corretamente 83.33% deles.


In [ ]:
print(classification_report(y_test, y_pred, target_names=['Sem Doença', 'Com Doença']))
#Compare Precision e Recall. Um valor alto em um pode significar um valor baixo no outro,
#dependendo do threshold e do objetivo do problema. Para detecção de doenças, Recall costuma ser muito importante.

              precision    recall  f1-score   support

  Sem Doença       0.87      0.94      0.90        48
  Com Doença       0.92      0.83      0.88        42

    accuracy                           0.89        90
   macro avg       0.89      0.89      0.89        90
weighted avg       0.89      0.89      0.89        90



# AUC

In [ ]:
y_pred_proba = model.predict_proba(x_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

print(f"AUC (Area Under the Curve): {roc_auc:.4f}")
print("Interpretação: Um AUC de 1.0 significa um modelo perfeito, 0.5 significa aleatório. "
      f"Nosso modelo tem uma boa capacidade de discriminação com AUC de {roc_auc:.4f}.")

AUC (Area Under the Curve): 0.9459
Interpretação: Um AUC de 1.0 significa um modelo perfeito, 0.5 significa aleatório. Nosso modelo tem uma boa capacidade de discriminação com AUC de 0.9459.
